# Map ChEMBL targets and get curve data

### Configuration

In [1]:
# ChEMBL connection...

engine = create_engine(open('database.txt').read().strip())

### Load list of unique symbols 

In [2]:
symbols = pd.read_pickle('unique_symbols.pkl')

symbols.shape

(215, 3)

<a name="target_mapping"></a>
## Get ChEMBL targets for symbols

See SQL file '[tt_chembl_targets](SQL/tt_chembl_targets.sql)' for details.

Note that protein families are _not_ included in the target mapping, only single proteins and protein complexes.

In [3]:
print(open('SQL/tt_chembl_targets.sql').read())

-- drop table tt_chembl_targets;

--

create table tt_chembl_targets
as
select
    x.symbol
  , x.approved_name
  , x.targets
  , row_number() over (partition by x.symbol, d.tax_id order by d.target_type, d.pref_name, d.chembl_id) as n_target
  , d.chembl_id
  , d.target_type
  , d.pref_name
  , case d.tax_id when 9606 then 'Human' when 10116 then 'Rat' end as species
  , 0 as exclude
from
  tt_symbols x
  join chembl_20_app.component_synonyms  a on x.symbol = upper(a.component_synonym)
  join chembl_20_app.component_sequences b on a.component_id = b.component_id
  join chembl_20_app.target_components   c on b.component_id = c.component_id
  join chembl_20_app.target_dictionary   d on c.tid          = d.tid
where
    a.syn_type = 'GENE_SYMBOL'
and d.target_type in ('SINGLE PROTEIN', 'PROTEIN COMPLEX')
and d.tax_id in (9606, 10116)
order by
    x.symbol
  , d.tax_id
  , n_target
;

----------------------------------------------------------------------------------------------------
-- En

In [4]:
targets = pd.read_sql_table('tt_chembl_targets', engine)

targets.shape

(377, 9)

In [5]:
HTML(targets.to_html())

,symbol,approved_name,targets,n_target,chembl_id,target_type,pref_name,species,exclude
0,ABCB1,"ATP-binding cassette, sub-family B (MDR/TAP), member 1","MDR1;7|MDR1;8|ATP-binding cassette, sub-family B (MDR/TAP), member 1;11",1,CHEMBL4302,SINGLE PROTEIN,P-glycoprotein 1,Human,0
1,ABCB1,"ATP-binding cassette, sub-family B (MDR/TAP), member 1","MDR1;7|MDR1;8|ATP-binding cassette, sub-family B (MDR/TAP), member 1;11",1,CHEMBL1075229,SINGLE PROTEIN,Multidrug resistance protein 1,Rat,0
2,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11",BSEP;6|BSEP;7|BSEP;8,1,CHEMBL6020,SINGLE PROTEIN,Bile salt export pump,Human,0
3,ABCB11,"ATP-binding cassette, sub-family B (MDR/TAP), member 11",BSEP;6|BSEP;7|BSEP;8,1,CHEMBL2073674,SINGLE PROTEIN,Bile salt export pump,Rat,0
4,ABCB4,"ATP-binding cassette, sub-family B (MDR/TAP), member 4",MDR3;7|MDR3;8,1,CHEMBL1743129,SINGLE PROTEIN,Multidrug resistance protein 3,Human,0
5,ABCB4,"ATP-binding cassette, sub-family B (MDR/TAP), member 4",MDR3;7|MDR3;8,1,CHEMBL2073706,SINGLE PROTEIN,Multidrug resistance protein 3,Rat,0
6,ABCC2,"ATP-binding cassette, sub-family C (CFTR/MRP), member 2","MRP2;7|MRP2;8|ATP-binding cassette, sub-family C (CFTR/MRP), member 2;11",1,CHEMBL5748,SINGLE PROTEIN,Canalicular multispecific organic anion transporter 1,Human,0
7,ABCC2,"ATP-binding cassette, sub-family C (CFTR/MRP), member 2","MRP2;7|MRP2;8|ATP-binding cassette, sub-family C (CFTR/MRP), member 2;11",1,CHEMBL2073676,SINGLE PROTEIN,Canalicular multispecific organic anion transporter 1,Rat,0
8,ABCC3,"ATP-binding cassette, sub-family C (CFTR/MRP), member 3",MRP3;7|MRP3;8,1,CHEMBL5918,SINGLE PROTEIN,Canalicular multispecific organic anion transporter 2,Human,0
9,ABCC3,"ATP-binding cassette, sub-family C (CFTR/MRP), member 3",MRP3;7|MRP3;8,1,CHEMBL2073682,SINGLE PROTEIN,Canalicular multispecific organic anion transporter 2,Rat,0


### Save/restore

In [6]:
targets.to_pickle('chembl_targets.pkl')

In [7]:
# targets = pd.read_pickle('chembl_targets.pkl')

# targets.shape

### Symbols for which no ChEMBL target exists

_N.B._ Only Human or Rat targets are currently considered.

In [8]:
# No. of symbols/genes for which a target exists in ChEMBL...

found = targets['symbol'].unique()

len(found) # N.B. 215 unique symbols input: see above

195

_I.e._ there are 195 symbols/genes (out of 215) for which a Human or Rat target exists in ChEMBL. Thus, 20 symbols are 'lost' at this stage.

Those symbols **_not_** found are shown below...

In [9]:
not_found = symbols[[x not in found for x in symbols.symbol]]

not_found

,symbol,approved_name,targets
symbol,,,
ABCG5,ABCG5,"ATP-binding cassette, sub-family G (WHITE), member 5","((ABCG5, 7),)"
ABCG8,ABCG8,"ATP-binding cassette, sub-family G (WHITE), member 8","((ABCG8, 7),)"
ATP1A1,ATP1A1,"ATPase, Na+/K+ transporting, alpha 1 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1A2,ATP1A2,"ATPase, Na+/K+ transporting, alpha 2 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1A3,ATP1A3,"ATPase, Na+/K+ transporting, alpha 3 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1A4,ATP1A4,"ATPase, Na+/K+ transporting, alpha 4 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1B1,ATP1B1,"ATPase, Na+/K+ transporting, beta 1 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1B2,ATP1B2,"ATPase, Na+/K+ transporting, beta 2 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"
ATP1B3,ATP1B3,"ATPase, Na+/K+ transporting, beta 3 polypeptide","((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))"


These 20 'lost' symbols correspond to 10 (nonredundant) targets, which are mostly ion channels and transporters...

In [10]:
def f(target, group):
    
    symbols = group['symbol'].values
        
    return {'targets': target, 'symbols': symbols, 'count': len(symbols)}
    
pd.DataFrame([f(x, y) for x, y in sorted(not_found.groupby('targets'), key=lambda x: float(x[0][0][1]))], columns=['targets', 'symbols', 'count']).sort('count', ascending=False).reset_index(drop=True)

,targets,symbols,count
0,"((ATPase (Na+/K+), 3), (ATPase (Na+/K+), 10))","[ATP1A1, ATP1A2, ATP1A3, ATP1A4, ATP1B1, ATP1B2, ATP1B3, ATP1B4]",8
1,"((IKP, 4),)","[KCNK1, KCNK4, KCNK6]",3
2,"((Ito,s, 4),)","[KCNA7, KCNC4]",2
3,"((ANF, 2.2),)",[NPR2],1
4,"((IK1, 4),)",[KCNJ12],1
5,"((OSTα/OSTβ, 6), (OSTα/β, 8))",[SLC51B],1
6,"((ABCG5, 7),)",[ABCG5],1
7,"((ABCG8, 7),)",[ABCG8],1
8,"((AE2, 7),)",[SLC4A2],1
9,"((FIC1, 7),)",[ATP8B1],1


<a name="get_curve_data"></a>
## Get curve data for ChEMBL Targets

See SQL files '[tt_curve_data_v1](SQL/tt_curve_data_v1.sql)' and '[tt_curve_data_v2](SQL/tt_curve_data_v2.sql)' for details of how the tables are created.

See '[tt_curve_data_v1_vs_v2](supplementary/tt_curve_data_v1_vs_v2.ipynb)' for a comparison of the two versions.

_N.B._ This step is performed before the final curation of the target list (see next notebook) as the count/activity information is used at that stage.

In [12]:
print(open('SQL/tt_curve_data_v1.sql').read())

-- drop table tt_curve_data_v1;

--

-- Note that the join to the 'compound_structures' table guarantees that a structure is available and that inorganics and peptides are excluded.

create table tt_curve_data_v1 as
select
    x.symbol
  , x.approved_name
  , x.species
  , x.chembl_id as target_chemblid
  , a.pref_name
  , a.target_type
  , b.chembl_id as assay_chemblid
  , b.description
  , j.chembl_id as parent_cmpd_chemblid
  , g.canonical_smiles as smiles
  , h.mw_freebase as amw
  , h.heavy_atoms as nat
  , i.chembl_id as cmpd_chemblid
  , c.activity_id
  , c.standard_type
  , c.standard_relation
  , c.standard_value
  , c.standard_units
  , c.pchembl_value
  , c.activity_comment
  , c.data_validity_comment
  , c.potential_duplicate
  , d.compound_key
  , c.published_type
  , c.published_relation
  , c.published_value
  , c.published_units
  , e.chembl_id as doc_chemblid
  , e.pubmed_id
  , case when e.journal is not null then e.journal || ', v. ' || e.volume || ', p. ' || e.first

In [14]:
print(open('SQL/tt_curve_data_v2.sql').read())

-- drop table tt_curve_data_v2;

--

-- Note that the join to the 'compound_structures' table guarantees that a structure is available and that inorganics and peptides are excluded.

create table tt_curve_data_v2 as
select
    x.symbol
  , x.approved_name
  , x.species
  , x.chembl_id as target_chemblid
  , a.pref_name
  , a.target_type
  , b.chembl_id as assay_chemblid
  , b.description
  , j.chembl_id as parent_cmpd_chemblid
  , g.canonical_smiles as smiles
  , h.mw_freebase as amw
  , h.heavy_atoms as nat
  , i.chembl_id as cmpd_chemblid
  , c.activity_id
  , c.standard_type
  , c.standard_relation
  , c.standard_value
  , c.standard_units
  , c.pchembl_value
  , c.activity_comment
  , c.data_validity_comment
  , c.potential_duplicate
  , d.compound_key
  , c.published_type
  , c.published_relation
  , c.published_value
  , c.published_units
  , e.chembl_id as doc_chemblid
  , e.pubmed_id
  , case when e.journal is not null then e.journal || ', v. ' || e.volume || ', p. ' || e.first